In [1]:
import QuantLib as ql

class DCrossCurrencySwap:    
    def __init__(self, evDate, effective, maturity, position, CCYComm, CommDC, 
                 CommIdx, CommFreq, CommSpread, InitCommNPA, FinalCommNPA, 
                 CCYTerm, TermDC, TermFreq, TermRate, InitTermNPA, FinalTermNPA, 
                 FXSpot, h_discTSComm, h_discTSTerm):
        self._ValueDate = evDate
        self._EffectiveDate = effective
        self._Maturity = maturity        
        if position == 'Long':          
            self._Position = 1.0
        else:
            self._Position = -1.0

        self._CCYComm = CCYComm
        self._CommDC = CommDC
        self._CommIdx = CommIdx
        self._InitCommNPA = InitCommNPA
        self._FinalCommNPA = FinalCommNPA
        self._CommSpread = CommSpread
        self._CommFreq = CommFreq

        self._CCYTerm = CCYTerm
        self._TermDC = TermDC
        self._InitTermNPA = InitTermNPA
        self._FinalTermNPA = FinalTermNPA
        self._TermRate = TermRate
        self._TermFreq = TermFreq

        self._FXSpot = FXSpot
        self._H_discTSComm = h_discTSComm
        self._H_discTSTerm = h_discTSTerm

    def NPV(self):
        _DGR = ql.DateGeneration.Backward
        _BDC = ql.Unadjusted
        _Cal = ql.NullCalendar()
        schTerm = ql.Schedule(self._EffectiveDate, self._Maturity, 
                              ql.Period(3, ql.Months), _Cal, _BDC, _BDC, _DGR, False)
        tenor = int((self._Maturity.year() - self._EffectiveDate.year()) 
                    * 12 / int(self._TermFreq))
        ListCoupon = list()
        for j in range(tenor):
            ListCoupon.append(self._TermRate)
            
        VecCoupon = ql.DoubleVector(ListCoupon)
        TermFaceAmt = self._FinalTermNPA
        TermBond = ql.FixedRateBond(0, TermFaceAmt, schTerm, VecCoupon, self._TermDC, 
                                    BDC, TermFaceAmt)
        TermBondEngine = ql.DiscountingBondEngine(self._H_discTSTerm)
        TermBond.setPricingEngine(TermBondEngine)
        TermValue = TermBond.dirtyPrice()
        
        schComm = ql.Schedule(self._EffectiveDate, self._Maturity, 
                              ql.Period(3, ql.Months), _Cal, _BDC, _BDC, _DGR, False)
        tenor = int((self._Maturity.year() - self._EffectiveDate.year()) 
                    * 12 / int(self._CommFreq))
        ListSpread = list()
        ListGearing = list()
        for j in range(tenor):
            ListSpread.append(self._CommSpread)
            ListGearing.append(1.0)
        VecSpread = ql.DoubleVector(ListSpread)
        VecGearing = ql.DoubleVector(ListGearing)
        CommFaceAmt = self._FinalCommNPA
        CommBond = ql.FloatingRateBond(0, CommFaceAmt, schComm, self._CommIdx, 
                                       self._CommDC, _BDC, 2, VecGearing, VecSpread)
        CommBondEngine = ql.DiscountingBondEngine(self._H_discTSComm)
        CommBond.setPricingEngine(CommBondEngine)
        CommValue = CommBond.dirtyPrice()
        
        if(evDate < self._EffectiveDate):
            dfComm = self._H_discTSComm.discount(self._EffectiveDate)
            dfTerm = self._H_discTSTerm.discount(self._EffectiveDate)
            initValue = (self._InitTermNPA * dfTerm 
                         - self._FXSpot * self._InitCommNPA * dfComm)
            self._Value = self._Position * (CommValue * self._FXSpot - TermValue 
                                            + initValue)
        else:
            self._Value = self._Position * (CommValue * self._FXSpot - TermValue)
        return self._Value

In [2]:
settings = ql.Settings.instance()
evDate = ql.Date(10, 7, 2011)
settings.setEvaluationDate(evDate)
Cal = ql.NullCalendar()
DC365 = ql.Actual365Fixed()
DC360 = ql.Actual360()
settlementDays = 0
BDC = ql.Unadjusted
Cal = ql.NullCalendar()
refDate = Cal.advance(evDate, settlementDays, ql.Days, ql.Following, False)

In [3]:
f = 0.01
discTSComm = ql.FlatForward(settlementDays, Cal, f, DC360, ql.Continuous, ql.Annual)
h_discTSComm = ql.YieldTermStructureHandle(discTSComm)

In [4]:
r = 0.02
discTSTerm = ql.FlatForward(settlementDays, Cal, r, DC365, ql.Continuous, ql.Annual)
h_discTSTerm = ql.YieldTermStructureHandle(discTSTerm)

In [5]:
Effective = ql.Date(15, 7, 2011)
Maturity = ql.Date(15, 7, 2013)
Position = 'Long'
CCYComm = 'USD'
CCYTerm = 'TWD'
FXSpot = 27.9

In [6]:
P3M = ql.Period(3, ql.Months)
USD = ql.USDCurrency()
EOM = False
FixingDays = 2
USDLIBOR3M = ql.IborIndex("LIBOR", P3M, FixingDays, USD, Cal, BDC, EOM, DC360, 
                          h_discTSComm)
CommFreq = 3
CommSpread = 0
TermFreq = 3
FixedRate = 0.0055

In [7]:
CCS = DCrossCurrencySwap(evDate, Effective, Maturity, Position, CCYComm, DC360,
                         USDLIBOR3M, CommFreq, CommSpread, 1000000.0, 1000000.0,
                         CCYTerm, DC365, TermFreq, FixedRate, 27891000.0, 28000000.0,
                         FXSpot, h_discTSComm, h_discTSTerm)
Value = CCS.NPV()
print(Value)

-26903238.279056296
